In [1]:
import pandas as pd
from Utils.DatasetClass import SubmissionDataset
import torchvision
from glob import glob
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import numpy as np
import pandas as pd

## Prepping the data

In [2]:
test_file_path = "./sample_submission.csv"

relationships_temp = pd.read_csv(test_file_path)
relationships = []

for x in relationships_temp['img_pair']:
    relationships.append(x.split('-'))

In [3]:
from torchvision import transforms

test_images_path = "./data/baseline/test/"

all_images = glob(test_images_path + "*.jpg")

test_person_to_images = {}

for x in all_images:

    if x.split("\\")[-1] not in test_person_to_images:
        test_person_to_images[x.split("\\")[-1]] = [x]

    else:
        test_person_to_images[x.split("\\")[-1]].append(x)

test_dataset = SubmissionDataset(relations = relationships, person_to_image= test_person_to_images)
testloader = DataLoader(test_dataset, batch_size= 1)

## Submission Testing Function definition

In [4]:
def submission_testing(model, testloader, device):
    model.eval()
    model.to(device)
    pred_list = []
    
    for batch in testloader:
        tensor1, tensor2 = batch
        tensor1, tensor2 = tensor1.to(device), tensor2.to(device)
        with torch.no_grad():
            output = model(tensor1, tensor2)
            pred = output>0.5
        pred_list.append(pred)

    return pred_list

## Instantiate model

In [5]:
from Utils.SiameseNet import SiameseNet
from Utils.OdinNet import Custom_Model
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_choice = 'OdinNet.pt'
MODEL_PATH = os.getcwd() + '\\models\\'+model_choice

model = Custom_Model()
model.load_state_dict(torch.load(MODEL_PATH, map_location= device))
model.eval()

c:\Users\ianch\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ianch\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ianch\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Custom_Model(
  (features): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
    

## Prediction starts

In [6]:
pred_list = submission_testing(model, testloader, device)

In [7]:
new_list = []

for y in pred_list:
    if y == False:
        new_list.append(0)
    else:
        new_list.append(1)

## Export Submission file

In [8]:
test_file_path = "./sample_submission.csv"

df = pd.read_csv(test_file_path)
df['is_related'] = new_list

output_file_name = 'csvSubmissions/final_submission_fixed.csv'

df.to_csv(output_file_name, index=False)